## code cell [1]

In [1]:
pip install ibapi numpy pandas matplotlib

Note: you may need to restart the kernel to use updated packages.


## test cell [1]

In [2]:
pip show ibapi numpy pandas matplotlib

Name: ibapi
Version: 9.81.1.post1
Summary: Official Interactive Brokers API
Home-page: https://interactivebrokers.github.io/tws-api
Author: IBG LLC
Author-email: dnastase@interactivebrokers.com
License: IB API Non-Commercial License or the IB API Commercial License
Location: c:\Users\avery\anaconda3\Lib\site-packages
Requires: 
Required-by: 
---
Name: numpy
Version: 1.26.4
Summary: Fundamental package for array computing in Python
Home-page: https://numpy.org
Author: Travis E. Oliphant et al.
Author-email: 
License: Copyright (c) 2005-2023, NumPy Developers.
All rights reserved.

Redistribution and use in source and binary forms, with or without
modification, are permitted provided that the following conditions are
met:

    * Redistributions of source code must retain the above copyright
       notice, this list of conditions and the following disclaimer.

    * Redistributions in binary form must reproduce the above
       copyright notice, this list of conditions and the following
 

## code cell [2]

In [ ]:
from ibapi.client import *
from ibapi.wrapper import *
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates

port = 4002

class TestApp(EClient, EWrapper):
    def __init__(self):
        EClient.__init__(self, self)
        self.data = []

    def nextValidId(self, orderId: OrderId):
        mycontract = Contract()
        mycontract.conId = 265598
        mycontract.exchange = "SMART"
        
        self.reqHistoricalData(
            reqId=123,
            contract=mycontract,
            endDateTime="",
            durationStr= "2 D",
            barSizeSetting = "1 hour",
            whatToShow= "TRADES",
            useRTH=0,
            formatDate=1,
            keepUpToDate=False,
            chartOptions=[],
        )

    def historicalData(self, reqId: int, bar: BarData):
        print("Historical Bar", bar)
        # Remove timezone information before appending
        date_without_tz = bar.date.split(' ')[0] + ' ' + bar.date.split(' ')[1]
        self.data.append([
            date_without_tz, bar.open, bar.high, bar.low, bar.close,
            bar.volume, bar.average, bar.barCount
        ])

    def historicalDataEnd(self, reqId: int, start: str, end: str):
        print("\nHistorical Data received \n")
        df = pd.DataFrame(self.data, columns=[
            'Date', 'Open', 'High', 'Low', 'Close', 'Volume', 'WAP', 'BarCount'
        ])
        # Convert 'Date' to datetime and set as index
        df['Date'] = pd.to_datetime(df['Date'], format='%Y%m%d %H:%M:%S')
        df.set_index('Date', inplace=True)
        print("Creating Data Frame...Printing DataFrame:\n")
        print(df)
        
        # Plot the data
        fig, ax1 = plt.subplots(figsize=(12, 8))

        # Plot OHLC data
        ax1.plot(df.index, df['Open'], label='Open', color='blue')
        ax1.plot(df.index, df['High'], label='High', color='green')
        ax1.plot(df.index, df['Low'], label='Low', color='red')
        ax1.plot(df.index, df['Close'], label='Close', color='black')
        ax1.set_ylabel('Price')
        ax1.legend(loc='upper left')

        # Create another y-axis for the volume data
        ax2 = ax1.twinx()
        ax2.fill_between(df.index, df['Volume'], color='gray', alpha=0.3, label='Volume')
        ax2.set_ylabel('Volume')
        ax2.legend(loc='upper right')

        # Format the x-axis to show the full date and time
        ax1.xaxis.set_major_formatter(mdates.DateFormatter('%Y%m%d%H:%M:%S'))
        plt.xticks(rotation=45)
        plt.gcf().autofmt_xdate()  # Auto format date for better visibility

        plt.title('Stock Price and Volume')
        plt.show()
        
        return super().historicalDataEnd(reqId, start, end)

    def error(self, reqId: TickerId, errorCode: int, errorString: str, advancedOrderRejectJson=""):
        print(reqId, errorCode, errorString, advancedOrderRejectJson)

app = TestApp()
app.connect("127.0.0.1", port, 1)
app.run()

-1 2104 Market data farm connection is OK:usfarm 
-1 2106 HMDS data farm connection is OK:ushmds 
-1 2158 Sec-def data farm connection is OK:secdefnj 
123 2176 Warning: Your API version does not support fractional share size rules. Please upgrade to a minimum version 163. Trimmed value 137.41 to 137 
Historical Bar Date: 20250226  09:00:00, Open: 246.620000, High: 246.720000, Low: 245.800000, Close: 245.890000, Volume: 137, Average: 246.253000, BarCount: 104
Historical Bar Date: 20250226  10:00:00, Open: 245.870000, High: 246.250000, Low: 245.620000, Close: 246.250000, Volume: 103, Average: 245.905000, BarCount: 86
Historical Bar Date: 20250226  11:00:00, Open: 246.210000, High: 246.260000, Low: 245.580000, Close: 245.600000, Volume: 81, Average: 245.899000, BarCount: 57
Historical Bar Date: 20250226  12:00:00, Open: 245.610000, High: 246.350000, Low: 245.610000, Close: 245.970000, Volume: 277, Average: 246.078000, BarCount: 186
Historical Bar Date: 20250226  13:00:00, Open: 246.220000

ValueError: time data "20250226 " doesn't match format "%Y%m%d %H:%M:%S", at position 0. You might want to try:
    - passing `format` if your strings have a consistent format;
    - passing `format='ISO8601'` if your strings are all ISO8601 but not necessarily in exactly the same format;
    - passing `format='mixed'`, and the format will be inferred for each element individually. You might want to use `dayfirst` alongside this.